https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [47]:
# Import libraries
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega

# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline

In [48]:
# display all rows of pandas data frame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-48-b6f59137b3d1>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [49]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '127.0.0.1'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'eicu'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [50]:

query = query_schema + """
SELECT *
FROM diagnosis
"""

diagnosis_df = pd.read_sql_query(query, con)
diagnosis_df.to_pickle("pickles/diagnosis.pkl")

query = query_schema + """
SELECT *
FROM patient
"""

patient_df = pd.read_sql_query(query, con)
patient_df.to_pickle("pickles/patient.pkl")


In [51]:
patient_df.shape

(200859, 29)

In [52]:
# join patient and diagnosis
patient_diagonsis_df = pd.merge(patient_df, diagnosis_df, on="patientunitstayid")

In [53]:
pd.DataFrame.head(patient_diagonsis_df)

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid,diagnosisid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,3596,Death,Expired,002-34851,4222318,False,72,cardiovascular|chest pain / ASHD|coronary artery disease|known,"414.00, I25.10",Other
1,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,3596,Death,Expired,002-34851,3370568,True,118,cardiovascular|ventricular disorders|cardiomyopathy,,Other
2,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,3596,Death,Expired,002-34851,4160941,False,72,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
3,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,3596,Death,Expired,002-34851,4103261,True,118,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
4,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,3596,Death,Expired,002-34851,3545241,True,118,cardiovascular|ventricular disorders|congestive heart failure,"428.0, I50.9",Other


In [54]:
sepsis_icd9 = '038.'
sepsis_df = patient_diagonsis_df[patient_diagonsis_df["icd9code"].str.contains(sepsis_icd9)]

In [55]:
sepsis_df.shape

(69830, 35)

In [56]:
sepsis_df = patient_diagonsis_df[patient_diagonsis_df["apacheadmissiondx"].str.contains("Sepsis")]
sepsis_df.shape

(527290, 35)

In [57]:
query = query_schema + """
SELECT *
FROM apacheapsvar
"""

apache_aps_df = pd.read_sql_query(query, con)
apache_aps_df.to_pickle("pickles/apacheapsvar.pkl")

In [58]:
sepsis_df = pd.merge(sepsis_df, apache_aps_df, on="patientunitstayid")
sepsis_df.shape
sepsis_df.to_pickle("pickles/sepsis_df.pkl")

In [59]:
query = query_schema + """
SELECT *
FROM hospital
"""

hospital_df = pd.read_sql_query(query, con)
hospital_df.to_pickle("pickles/hospital.pkl")

In [60]:
hospital_df.shape

(208, 4)

In [61]:
sepsis_df["hospitalid"].value_counts()

420    89878
122    21433
188    16779
142    11885
79     11464
394    10582
252    10047
167    8958 
391    8763 
417    8187 
73     8068 
365    7894 
140    7134 
243    7103 
419    6940 
226    6854 
157    6826 
176    6808 
199    6777 
400    5827 
271    5718 
110    5698 
227    5552 
416    5546 
388    5523 
392    5508 
390    5428 
141    5290 
449    5121 
165    4972 
382    4815 
148    4678 
440    4649 
386    4575 
458    4418 
248    4375 
435    4180 
154    4060 
92     3977 
353    3977 
384    3889 
281    3507 
434    3494 
283    3351 
206    3293 
424    3278 
277    3105 
155    2726 
171    2726 
244    2661 
403    2591 
208    2524 
407    2493 
421    2374 
405    2305 
396    2283 
280    2257 
444    2216 
85     2147 
397    2012 
66     1944 
357    1939 
202    1898 
399    1895 
95     1781 
452    1772 
63     1769 
272    1758 
108    1757 
253    1757 
429    1643 
360    1558 
381    1554 
358    1522 
436    1511 
183    1447 
300    1430 

In [62]:
# 89878 people
big_hospital_id = 420 

# 11885 people
medium_hospital_id = 142

# 5428 people
small_hospital_id = 390

In [63]:
big_hospital_df = sepsis_df[sepsis_df["hospitalid"] == big_hospital_id]
big_hospital_df.to_pickle("pickles/big_hospital.pkl")

medium_hospital_df = sepsis_df[sepsis_df["hospitalid"] == medium_hospital_id]
medium_hospital_df.to_pickle("pickles/medium_hospital.pkl")

small_hospital_df = sepsis_df[sepsis_df["hospitalid"] == small_hospital_id]
small_hospital_df.to_pickle("pickles/small_hospital.pkl")


In [64]:
print("big hospital dims:\t", big_hospital_df.shape)
print("medium hospital dims:\t", medium_hospital_df.shape)
print("small hospital dims:\t", small_hospital_df.shape)

big hospital dims:	 (89878, 60)
medium hospital dims:	 (11885, 60)
small hospital dims:	 (5428, 60)


In [68]:
three_hospital_df = sepsis_df[(sepsis_df["hospitalid"] == big_hospital_id) |
                             (sepsis_df["hospitalid"] == medium_hospital_id) |
                             (sepsis_df["hospitalid"] == small_hospital_id)]
three_hospital_df.to_pickle("pickles/three_hospital.pkl")
three_hospital_df.shape

(107191, 60)

In [69]:
89878 +11885 + 5428

107191